In [1]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout


In [164]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.neighbors import KNeighborsClassifier as KNN
import talib as ta
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import seaborn as sm
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.model_selection import train_test_split as split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
import random
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.utils import resample
from itertools import combinations
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from sklearn.linear_model import Ridge

In [165]:
def rnn(shape,numberOfFeatures):
    regressor = Sequential()

    # %%
    regressor.add(LSTM(units = 50, return_sequences = True,input_shape = (shape,numberOfFeatures))) 
    regressor.add(Dropout(rate = 0.2))
    #regressor.add(Dense(units=16,activation = 'relu',input_shape = (xTrain.shape[1],numberOfFeatures)))

    # %%
    regressor.add(LSTM(units = 50, return_sequences = True)) 
    regressor.add(Dropout(rate = 0.2))
    #regressor.add(Dense(units=32,activation = 'relu'))

    # %%
    regressor.add(LSTM(units = 50, return_sequences = True))
    regressor.add(Dropout(rate = 0.2))
    #regressor.add(Dense(units=16,activation = 'relu'))

    # %%
    regressor.add(LSTM(units = 50))
    regressor.add(Dropout(rate = 0.2))
    # Last Layer
    regressor.add(Dense(units=1))#output layer, default since this is regression not classfition 

    # %% [markdown]
    # ## Adding output layer

    # %%
    regressor.compile(optimizer='adam',loss='mean_squared_error',metrics='accuracy')
    return regressor
            


In [166]:
def getVariables(variablesToInclude,dataTrain):
    trainingDataPoints = round(len(dataTrain) * .4)
    numberOfFeatures = len(variablesToInclude)
    newTrain = dataTrain.iloc[:,variablesToInclude]
    data = dataTrain[:trainingDataPoints]['Close']
    if 'Close' in newTrain.columns:
        newTrain["Close"] = newTrain['Close'].shift(1)
    trainingSet = newTrain.iloc[:,0:numberOfFeatures].values #convert to numpy to train RNN
    ySet = data.astype(float).values.reshape(-1, 1)

    # %% [markdown]
    # ## Feature Scaling

    # %%
    # Use normalization x - min(x) / max(min) - min(x)
    sc = MinMaxScaler(feature_range=(0,1)) # all values between 0 and 1
    ySC = MinMaxScaler(feature_range=(0,1))
    ySCSet = ySC.fit_transform(ySet)
    scaleTrainingSet = sc.fit_transform(trainingSet)

    # %% [markdown]
    # ## Create a data structure woth 60 timesteps and 1 output

    # %%
    # Look at the 60 previous timesteps to predict this timestep
    xTrain = []
    yTrain = []
    for i in range(window+33,trainingDataPoints):
        xTrain.append(scaleTrainingSet[i-window:i])
        yTrain.append(ySCSet[i])

    # %%
    # convert xtrain and yTrain to numpy for RNN
    xTrain, yTrain = np.array(xTrain), np.array(yTrain)

    # %% [markdown]
    # ## Reshaping the data
    inputs = newTrain.iloc[trainingDataPoints+1:,0:numberOfFeatures].values #convert to numpy to train RNN
    # %%
    xTest = []
    for i in range(window+33,len(inputs)):
        xTest.append(inputs[i-window:i])
    xTest = np.array(xTest)
    xTest = np.reshape(xTest, (xTest.shape[0],xTest.shape[1],numberOfFeatures)) #batchsize, inputSize, numberOfFeatures
    real = dataTrain[trainingDataPoints+1:]['Close']

    xTrain = np.reshape(xTrain, (xTrain.shape[0],xTrain.shape[1],numberOfFeatures)) #batchsize, inputSize, numberOfFeatures
    return xTrain,yTrain,xTest,real,ySC



In [189]:
def evalProfit(yTest,real,ySC):
    stocksOwned = {}
    predictedPrice = ySC.inverse_transform(yTest)
    realStockPrice = real[window-1:].values
    startingValue = 500_000/505
    liquidValue = startingValue

    for i in range(len(predictedPrice)):
        if realStockPrice[i] < predictedPrice[i] and 'GOOGL' not in stocksOwned:
            stocksOwned['GOOGL'] = (realStockPrice[i], liquidValue/realStockPrice[i])
            liquidValue -= liquidValue/realStockPrice[i] * realStockPrice[i]
        elif 'GOOGL' in stocksOwned and stocksOwned['GOOGL'][0] < realStockPrice[i]:
            liquidValue += stocksOwned['GOOGL'][1] * realStockPrice[i]
            stocksOwned.pop('GOOGL')

    profit = (liquidValue - startingValue)
    return profit


In [168]:
rsiPeriod = 14
adxPeriod = 14
bollingerBandWindow = 20
numberOfFeatures = 11
trainingDataPoints = 3500
window = 60
LE = LabelEncoder()
linReg = LinearRegression()

In [169]:
dataTrain = pd.read_csv("/Users/ishan/Coding/Wpi/StockMarketSimulationIQP/Datasets/30y_stock_csvs/TSLA.csv") 
dataTrain.rename(columns = {'CLOSE':'Close'}, inplace = True)
dataTrain.rename(columns = {'HIGH':'High'}, inplace = True)
dataTrain.rename(columns = {'LOW':'Low'}, inplace = True)
dataTrain.rename(columns = {'VOLUME':'Volume'}, inplace = True)
dataTrain = dataTrain.drop(columns='Datetime')
dataTrain.describe()

,OPEN,High,Low,Close,Volume
count,2929.000000,2929.000000,2929.000000,2929.000000,2.929000e+03
mean,132.062589,134.926862,128.976899,132.081496,3.136157e+07
std,241.350639,246.788346,235.328789,241.270193,2.810150e+07
min,3.228000,3.326000,2.996000,3.160000,5.925000e+05
25%,18.844000,19.524000,17.822001,18.893999,1.304250e+07
50%,46.358002,47.146000,45.625999,46.334000,2.502050e+07
75%,67.519997,68.779999,66.358002,67.403999,3.993800e+07
max,1234.410034,1243.489990,1217.000000,1229.910034,3.046940e+08


In [170]:
#import csv
dataTrain['CloseEdited'] = dataTrain['Close'].astype(float)
dataTrain["RSI"] = ta.RSI(dataTrain['CloseEdited'],rsiPeriod)
dataTrain["ADX"] = ta.ADX(dataTrain['High'],dataTrain['Low'],dataTrain['CloseEdited'],adxPeriod)
fastk, fastd = ta.STOCHF(dataTrain['High'],dataTrain['Low'],dataTrain['CloseEdited'])
dataTrain['fastd'] = fastd
dataTrain['fastk'] = fastk
macd, macdsignal, macdhist = ta.MACD(dataTrain['CloseEdited'])
dataTrain['macd'] = macd
dataTrain['macdsignal'] = macdsignal
dataTrain['macdhist'] = macdhist
upper,middle,lower = ta.BBANDS(dataTrain['CloseEdited'])
dataTrain['bb_lowerband'] = lower
dataTrain['bb_middleband'] = middle
dataTrain['bb_upperband'] = upper
newTrain = pd.DataFrame()
newTrain["Close"] = dataTrain['Close'].astype(float)
newTrain["Close"][1:] = newTrain["Close"][0:-1]
newTrain['Volume'] = dataTrain['Volume'].astype(float)
newTrain['RSI'] = dataTrain['RSI'].fillna(0)
newTrain["ADX"] = dataTrain["ADX"].fillna(0)
newTrain['fastd'] = dataTrain['fastd'].fillna(0)
newTrain['fastk'] = dataTrain['fastk'].fillna(0)
newTrain['macd'] = dataTrain['macd'].fillna(0)
newTrain['macdsignal'] = dataTrain['macdsignal'].fillna(0)
newTrain['macdhist'] = dataTrain['macdhist'].fillna(0)
newTrain['bb_lowerband'] = dataTrain['bb_lowerband'].fillna(0)
newTrain['bb_middleband'] = dataTrain['bb_middleband'].fillna(0)
newTrain['bb_upperband'] = dataTrain['bb_upperband'].fillna(0)
trainingSet = newTrain.iloc[:,0:numberOfFeatures].values #convert to numpy to train RNN
trainingSet
trainingDataPoints = round(len(newTrain) * .4)


In [171]:
ySet = dataTrain['Close'].astype(float).values
ySet

array([  4.77799988,   4.76599979,   4.3920002 , ..., 904.54998779,
       860.        , 864.32952881])

In [172]:
x = newTrain
y = ySet.ravel()
y = LE.fit_transform(y).astype(int)


In [161]:


print(x.shape)
print(y.shape)



knn = KNN(n_neighbors = 4)



sfs1 = SFS(knn, 
           k_features=12, 
           forward=True, 
           floating=False, 
           verbose=2,
           scoring='accuracy',
           cv=0)

sfs1 = sfs1.fit(x, y)

(2929, 12)
(2929,)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    1.6s finished

[2022-03-07 22:17:50] Features: 1/12 -- score: 0.2543530215090475[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    1.9s finished

[2022-03-07 22:17:52] Features: 2/12 -- score: 0.25230454079890746[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s

STOPPING EARLY DUE TO KEYBOARD INTERRUPT...

In [127]:
sfs1.subsets_

{1: {'feature_idx': (5,),
  'cv_scores': array([0.25435302]),
  'avg_score': 0.2543530215090475,
  'feature_names': ('fastk',)},
 2: {'feature_idx': (2, 5),
  'cv_scores': array([0.25230454]),
  'avg_score': 0.25230454079890746,
  'feature_names': ('RSI', 'fastk')},
 3: {'feature_idx': (2, 4, 5),
  'cv_scores': array([0.25230454]),
  'avg_score': 0.25230454079890746,
  'feature_names': ('RSI', 'fastd', 'fastk')},
 4: {'feature_idx': (2, 3, 4, 5),
  'cv_scores': array([0.25503585]),
  'avg_score': 0.25503584841242743,
  'feature_names': ('RSI', 'ADX', 'fastd', 'fastk')},
 5: {'feature_idx': (1, 2, 3, 4, 5),
  'cv_scores': array([0.24820758]),
  'avg_score': 0.24820757937862753,
  'feature_names': ('Volume', 'RSI', 'ADX', 'fastd', 'fastk')},
 6: {'feature_idx': (0, 1, 2, 3, 4, 5),
  'cv_scores': array([0.24820758]),
  'avg_score': 0.24820757937862753,
  'feature_names': ('Close', 'Volume', 'RSI', 'ADX', 'fastd', 'fastk')},
 7: {'feature_idx': (0, 1, 2, 3, 4, 5, 6),
  'cv_scores': array([

In [128]:
sfs1.k_feature_names_

('Close',
 'Volume',
 'RSI',
 'ADX',
 'fastd',
 'fastk',
 'macd',
 'macdsignal',
 'macdhist',
 'bb_lowerband',
 'bb_middleband',
 'bb_upperband')

In [191]:
def best_subset(X, y, max_size=8, cv=5):
    n_features = X.shape[1]
    subsets = (combinations(range(n_features), k + 1) for k in range(min(n_features, max_size)))
    
    best_size_subset = []
    for subsets_k in subsets:  # for each list of subsets of the same size
        best_score = -np.inf
        best_subset = None
        for subset in subsets_k: # for each subset
            xTrain,yTrain,xTest,real,ySC = getVariables(list(subset),dataTrain)
            with HiddenPrints():
                estimator = rnn(xTrain.shape[1],len(list(subset)))
                estimator.fit(xTrain,yTrain,epochs=1,batch_size=32)
            # get the subset with the best score among subsets of the same size
            yTest = estimator.predict(xTest)
            score = evalProfit(yTest,real,ySC)
            if score > best_score:
                best_score, best_subset = score, subset
        # to compare subsets of different sizes we must use CV
        # first store the best subset of each size
        best_size_subset.append(best_subset)
        
    # compare best subsets of each size
    best_score = -np.inf
    best_subset = None
    list_scores = []
    for subset in best_size_subset:
        score = cross_val_score(estimator, X.iloc[:, list(subset)], y, cv=cv).mean()
        list_scores.append(score)
        if score > best_score:
            best_score, best_subset = score, subset

    return best_subset, best_score, best_size_subset, list_scores


In [192]:

best_subsets, best_score, best_size_subset, list_scores = best_subset(x,y,max_size=12)
best_score

[0]
34/34 [==============================] - 14s 123ms/step - loss: 0.0947 - accuracy: 0.0000e+00
[1]
34/34 [==============================] - 11s 123ms/step - loss: 0.0974 - accuracy: 0.0000e+00
[2]
34/34 [==============================] - 11s 122ms/step - loss: 0.0688 - accuracy: 0.0000e+00
[3]


/Users/ishan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


34/34 [==============================] - 11s 121ms/step - loss: 0.1121 - accuracy: 0.0000e+00
[4]
34/34 [==============================] - 11s 123ms/step - loss: 0.0813 - accuracy: 0.0000e+00
[5]
34/34 [==============================] - 11s 129ms/step - loss: 0.0805 - accuracy: 9.2678e-04
[6]
34/34 [==============================] - 11s 128ms/step - loss: 0.1047 - accuracy: 0.0000e+00
[7]
34/34 [==============================] - 12s 133ms/step - loss: 0.1015 - accuracy: 0.0000e+00
[8]
34/34 [==============================] - 12s 129ms/step - loss: 0.1043 - accuracy: 0.0000e+00
[9]
34/34 [==============================] - 13s 151ms/step - loss: 0.1078 - accuracy: 0.0000e+00
[10]
34/34 [==============================] - 12s 117ms/step - loss: 0.1107 - accuracy: 0.0000e+00
[11]
34/34 [==============================] - 10s 92ms/step - loss: 0.1118 - accuracy: 0.0000e+00
[0, 1]
34/34 [==============================] - 12s 134ms/step - loss: 0.0853 - accuracy: 0.0000e+00
[0, 2]
34/34 [======

/Users/ishan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


34/34 [==============================] - 13s 142ms/step - loss: 0.0796 - accuracy: 0.0000e+00
[0, 4]
34/34 [==============================] - 15s 139ms/step - loss: 0.0729 - accuracy: 0.0000e+00
[0, 5]
34/34 [==============================] - 13s 142ms/step - loss: 0.0930 - accuracy: 0.0000e+00
[0, 6]
34/34 [==============================] - 12s 130ms/step - loss: 0.1065 - accuracy: 0.0000e+00
[0, 7]
34/34 [==============================] - 12s 121ms/step - loss: 0.1052 - accuracy: 0.0000e+00
[0, 8]
34/34 [==============================] - 13s 129ms/step - loss: 0.1068 - accuracy: 0.0000e+00
[0, 9]
34/34 [==============================] - 11s 115ms/step - loss: 0.1041 - accuracy: 0.0000e+00
[0, 10]
34/34 [==============================] - 9s 79ms/step - loss: 0.1143 - accuracy: 0.0000e+00
[0, 11]
34/34 [==============================] - 7s 61ms/step - loss: 0.1103 - accuracy: 0.0000e+00
[1, 2]
34/34 [==============================] - 11s 74ms/step - loss: 0.0701 - accuracy: 0.0000e+00


...

/Users/ishan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


34/34 [==============================] - 6s 61ms/step - loss: 0.1019 - accuracy: 0.0000e+00
[0, 2, 3, 10, 11]


/Users/ishan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


34/34 [==============================] - 6s 65ms/step - loss: 0.0974 - accuracy: 0.0000e+00
[0, 2, 4, 5, 6]
34/34 [==============================] - 9s 110ms/step - loss: 0.1005 - accuracy: 9.2678e-04
[0, 2, 4, 5, 7]
34/34 [==============================] - 7s 64ms/step - loss: 0.0916 - accuracy: 0.0000e+00
[0, 2, 4, 5, 8]
34/34 [==============================] - 7s 57ms/step - loss: 0.1027 - accuracy: 0.0000e+00
[0, 2, 4, 5, 9]
34/34 [==============================] - 6s 49ms/step - loss: 0.0792 - accuracy: 0.0000e+00
[0, 2, 4, 5, 10]
34/34 [==============================] - 6s 57ms/step - loss: 0.0754 - accuracy: 0.0000e+00
[0, 2, 4, 5, 11]
34/34 [==============================] - 7s 60ms/step - loss: 0.0751 - accuracy: 0.0000e+00
[0, 2, 4, 6, 7]
34/34 [==============================] - 9s 79ms/step - loss: 0.0835 - accuracy: 0.0000e+00
[0, 2, 4, 6, 8]
34/34 [==============================] - 7s 61ms/step - loss: 0.0966 - accuracy: 0.0000e+00
[0, 2, 4, 6, 9]
34/34 [==================

KeyboardInterrupt: 

In [ ]:
best_subsets

(1, 2, 9)

In [ ]:
best_score

-15.731829932067578

In [13]:
best_size_subset

[(0,),
 (1, 9),
 (1, 2, 9),
 (1, 2, 9, 10),
 (1, 2, 6, 9, 11),
 (1, 2, 3, 6, 9, 10),
 (1, 2, 3, 4, 6, 9, 10),
 (1, 2, 3, 4, 5, 6, 9, 10)]

In [14]:
list_scores

[-1076.0527316093835,
 -1026.5690390234154,
 -1021.5094301344947,
 -1151.9652670735863,
 -1127.2203425471396,
 -1128.1094620628642,
 -1126.7496063606752,
 -1130.1779753557526]

In [193]:
best_score

-1021.5094301344947

In [196]:
subset

NameError: name 'subset' is not defined